In [1]:
import math
import os
import openpyxl
import platform
import pandas as pd

In [2]:
if platform.platform()[:3].lower() == 'mac':
    mac_m1 = True
elif platform.platform()[:3].lower() == 'win':
    mac_m1 = False
    
if mac_m1:
    path = '/Users/rainyseason/winston/Workspace/python/Pycharm Project/sinewave_analyze/Evaluation/'
    path_csv = path + 'tek_csv/'
    path_excel = path + 'tek_excel/'
    path_summary = path + 'summary/'
    path_information = path + 'test information/'
    path_kmon = path + 'kmon_csv'
else:
    path = 'D:/winston/OneDrive - (주)필드큐어/data_analyze/'
    path_csv = path + 'tek_csv/'
    path_excel = path + 'tek_excel/'
    path_summary = path + 'summary/'
    path_information = path + 'test information/'
    path_kmon = path + 'kmon_csv/'

In [3]:
summary = pd.ExcelFile(path_excel + 'summary.xlsx')
sheets = summary.sheet_names
print(sheets)
sheets = sheets[2:]
sheets

['summary', 'with kmon', 'with kmon Curr', 'with kmon Volt']


['with kmon Curr', 'with kmon Volt']

In [4]:
df_summary = summary.parse(sheet_name=sheets[0])

In [5]:
df_summary

,filename,Board,ohm,Ch,PWM,Volt[V],Curr[mA],Vpeak[V],Irms[mA],FFT I rms[mA],RF Curr Ch 1[mA],RF Curr Ch 1 deviation,RF Curr Ch 2[mA],RF Curr Ch 2 deviation,RF Curr Ch 3[mA],RF Curr Ch 3 deviation,RF Curr Ch 4[mA],RF Curr Ch 4 deviation
0,tek3166,RFAMP_03,500,Ch1,-,-,0.0,NaN,NaN,2.049818,0.186364,0.841830,0.154545,0.655555,0.179545,0.693461,0.147727,0.621166
1,tek3167,RFAMP_03,500,Ch1,-,-,2.5,NaN,NaN,7.702295,6.307143,2.831732,0.140476,0.579800,0.166667,0.604218,0.147619,0.587087
2,tek3168,RFAMP_03,500,Ch1,-,-,5.0,6.186629,8.577015,8.228101,6.364286,3.175962,0.142857,0.583212,0.173810,0.788595,0.152381,0.698232
3,tek3169,RFAMP_03,500,Ch1,-,-,7.5,6.386901,8.852768,8.464102,6.352381,2.280749,0.154762,0.624972,0.164286,0.717848,0.147619,0.698232
4,tek3170,RFAMP_03,500,Ch1,-,-,10.0,5.996488,8.363988,7.927740,6.326829,2.785191,0.153659,0.588407,0.180488,0.739793,0.141463,0.539901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,tek3613,RFAMP_03,500,Ch4,-,-,270.0,120.508501,163.980716,163.895338,2.206977,1.770499,0.318605,0.970634,1.862791,1.804687,156.481395,1.040032
448,tek3614,RFAMP_03,500,Ch4,-,-,272.5,120.725002,164.262956,164.170792,2.295122,1.464227,0.346341,0.990136,1.904878,1.576523,156.765854,1.201823
449,tek3615,RFAMP_03,500,Ch4,-,-,275.0,121.007233,164.757944,164.661168,2.300000,1.902379,0.314286,0.888322,1.933333,1.959916,157.069048,1.079861
450,tek3616,RFAMP_03,500,Ch4,-,-,277.5,121.190516,165.110732,165.012037,2.304762,1.413412,0.333333,0.917208,1.947619,1.665306,157.326190,1.216152


In [31]:
remain_columns = ['Board', 'ohm', 'Irms[mA]']
delete_columns = list(df_summary.columns)

if 'curr' in sheet.lower():
    remain_columns.insert(2, 'Curr[mA]')
if 'volt' in sheet.lower():
    remain_columns.append(2, 'Volt[V]')

df = pd.DataFrame(columns=remain_columns)

blank_value = []
for i in range(len(df.columns)):
    blank_value.append('')

max_length_ch = df_summary['Ch'].value_counts()
max_ch = max_length_ch.index[0]

max_length_ch = df_summary['Ch'].value_counts()
max_length_ch = max_length_ch.index[0]

start = 0
end = 0
before = ''
for i in range(len(df_summary)):
    if df_summary.at[i, 'Ch'] == max_ch and before == '':
        start = i
        before = df_summary.at[i, 'Ch']
    elif before != df_summary.at[i, 'Ch']:
        end = i - 1
        break
    elif before == df_summary.at[i, 'Ch'] and i == len(df_summary) - 1:
        end = i

In [32]:
for i in range(start, end + 1):
    df.loc[i - start] = blank_value

In [33]:
df

,Board,ohm,Curr[mA],Irms[mA]
0,,,,
1,,,,
2,,,,
3,,,,
4,,,,
...,...,...,...,...
108,,,,
109,,,,
110,,,,
111,,,,


In [43]:
for column in remain_columns:
    for i in range(start, end + 1):
        df.at[i - start, column] = df_summary.at[i, column]

In [46]:
df_summary['Ch'].unique()

array(['Ch1', 'Ch2', 'Ch3', 'Ch4'], dtype=object)

In [42]:
df_summary

,filename,Board,ohm,Ch,PWM,Volt[V],Curr[mA],Vpeak[V],Irms[mA],FFT I rms[mA],RF Curr Ch 1[mA],RF Curr Ch 1 deviation,RF Curr Ch 2[mA],RF Curr Ch 2 deviation,RF Curr Ch 3[mA],RF Curr Ch 3 deviation,RF Curr Ch 4[mA],RF Curr Ch 4 deviation
0,tek3166,RFAMP_03,500,Ch1,-,-,0.0,NaN,NaN,2.049818,0.186364,0.841830,0.154545,0.655555,0.179545,0.693461,0.147727,0.621166
1,tek3167,RFAMP_03,500,Ch1,-,-,2.5,NaN,NaN,7.702295,6.307143,2.831732,0.140476,0.579800,0.166667,0.604218,0.147619,0.587087
2,tek3168,RFAMP_03,500,Ch1,-,-,5.0,6.186629,8.577015,8.228101,6.364286,3.175962,0.142857,0.583212,0.173810,0.788595,0.152381,0.698232
3,tek3169,RFAMP_03,500,Ch1,-,-,7.5,6.386901,8.852768,8.464102,6.352381,2.280749,0.154762,0.624972,0.164286,0.717848,0.147619,0.698232
4,tek3170,RFAMP_03,500,Ch1,-,-,10.0,5.996488,8.363988,7.927740,6.326829,2.785191,0.153659,0.588407,0.180488,0.739793,0.141463,0.539901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,tek3613,RFAMP_03,500,Ch4,-,-,270.0,120.508501,163.980716,163.895338,2.206977,1.770499,0.318605,0.970634,1.862791,1.804687,156.481395,1.040032
448,tek3614,RFAMP_03,500,Ch4,-,-,272.5,120.725002,164.262956,164.170792,2.295122,1.464227,0.346341,0.990136,1.904878,1.576523,156.765854,1.201823
449,tek3615,RFAMP_03,500,Ch4,-,-,275.0,121.007233,164.757944,164.661168,2.300000,1.902379,0.314286,0.888322,1.933333,1.959916,157.069048,1.079861
450,tek3616,RFAMP_03,500,Ch4,-,-,277.5,121.190516,165.110732,165.012037,2.304762,1.413412,0.333333,0.917208,1.947619,1.665306,157.326190,1.216152


In [59]:
channels = []
start = []
end = []
before = ''
for i in range(len(df_summary)):
    if before == '':
        start.append(i)
        channels.append(df_summary.at[i, 'Ch'])
        before = df_summary.at[i, 'Ch']
    elif before != df_summary.at[i, 'Ch']:
        start.append(i)
        channels.append(df_summary.at[i, 'Ch'])
        end.append(i - 1)
        before = df_summary.at[i, 'Ch']
    elif before == df_summary.at[i, 'Ch'] and i == len(df_summary) - 1:
        end.append(i)

In [70]:
df

,Board,ohm,Curr[mA],Irms[mA]
0,RFAMP_03,500,0.0,NaN
1,RFAMP_03,500,2.5,NaN
2,RFAMP_03,500,5.0,8.577015
3,RFAMP_03,500,7.5,8.852768
4,RFAMP_03,500,10.0,8.363988
...,...,...,...,...
108,RFAMP_03,500,270.0,195.419882
109,RFAMP_03,500,272.5,195.938766
110,RFAMP_03,500,275.0,196.214217
111,RFAMP_03,500,277.5,196.704307


In [90]:
dict = {}

In [91]:
for ch in channels:
    dict.setdefault('Irms[mA]' + ch, [])

In [92]:
dict

{'Irms[mA]Ch1': [], 'Irms[mA]Ch2': [], 'Irms[mA]Ch3': [], 'Irms[mA]Ch4': []}

In [ ]:
for i in range(df_summary):
    